In [2]:
import os
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split


# Modified Images directory
image_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/Imagenette_LIME"

attack_label = 4  

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

class AttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))  

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB") 

        if self.transform:
            image = self.transform(image)

        return image, self.label

torch.manual_seed(42)  

attack_dataset = AttackDataset(image_dir=image_dir, label=attack_label, transform=transform)

total_samples = len(attack_dataset)

# Calculate split sizes
test_size = int(0.2 * total_samples)  # 10% for testing
train_size = total_samples - test_size  # Remaining for training

# Split the dataset
train_dataset, test_dataset = random_split(attack_dataset, [train_size, test_size])

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Training samples: {len(train_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")

# Create a DataLoader for the attack dataset
attack_loader = DataLoader(attack_dataset, batch_size=64, shuffle=True)

Training samples: 758
Test samples: 189


In [6]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [7]:
device = 'cuda'
model_path = "/home/j597s263/scratch/j597s263/Models/ConvModels/Conv_Imagenette(Defended).mod"
model = torch.load(model_path, weights_only=False, map_location=device)

In [8]:
# Evaluation loop for attack_loader
model.eval()
correct = 0
total = 0
test_loss = 0.0

criterion = nn.CrossEntropyLoss()


with torch.no_grad():
    for images, labels in attack_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Compute accuracy
attack_accuracy = 100 * correct / total
print(f"Attack Dataset Accuracy: {attack_accuracy:.2f}%")

Attack Dataset Accuracy: 13.09%
